In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

In [ ]:
"""
find relevent part in docs
load docs -> split into chunks -> embed -> vector store -> retriver (return chunks relevant to user question) -> search

stuff: add chunks to prompt
refine: update answers after anwering to every chunk
map_reduce: find relevant parts in each chunk, combine, and create a final answer
map_rerank: answer for each chunk while scoring, and create a final answer
"""

"""
if you want to search and use a whole file content
load docs -> split into chunks
"""

In [5]:
# load, split
splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, # max size of each chunk
    chunk_overlap=50, # overlap between chunks
)
splitter = CharacterTextSplitter.from_tiktoken_encoder( # .from_tiktoken_encoder(): count length of tokens (just how the model counts)
    separator="\n", # split standard
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")
docs = loader.load_and_split(text_splitter=splitter)

# embed
cache_dir = LocalFileStore("./.cache/")
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriever = vectorstore.as_retriever() # input: user question, output: list of relevant doc chunks

# search
llm = ChatOpenAI()

In [ ]:
## off-the-shelf chain
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)
chain.run("Describe Victory Mansions")

In [7]:
## customized chain using lcel

### stuff
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a helpful assistant. Answer questions using only the following context. 
            If you don't know the answer just say you don't know, don't make it up:
            \n
            \n
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)
chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm
chain.invoke(
    "Describe Victory Mansions"
)  # "Describe Victory Mansions" is used in retriever and prompt

### map_reduce
find_relevant_parts_in_chunk_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question.\
            Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {chunk}
            """,
        ),
        ("human", "{question}"),
    ]
)
find_relevant_parts_in_chunk_chain = find_relevant_parts_in_chunk_prompt | llm


def combine_relevant_parts(inputs):
    chunks = inputs["chunks"]
    question = inputs["question"]
    return "\n\n".join(
        find_relevant_parts_in_chunk_chain.invoke(
            {"chunk": chunk.page_content, "question": question}
        ).content
        for chunk in chunks
    )


combine_relevant_parts_chain = {
    "chunks": retriever,  # list of chunks
    "question": RunnablePassthrough(),
} | RunnableLambda(combine_relevant_parts)

final_answer_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {parts}
            """,
        ),
        ("human", "{question}"),
    ]
)
final_answer_chain = (
    {"parts": combine_relevant_parts_chain, "question": RunnablePassthrough()}
    | final_answer_prompt
    | llm
)
final_answer_chain.invoke("Describe Victory Mansions")

AIMessage(content='Victory Mansions is a rundown residential building where the protagonist, Winston Smith, lives in the novel "1984" by George Orwell. It has glass doors that let in gritty dust, a hallway that smells of boiled cabbage and old rag mats, and a colored poster of an enormous face with the caption "BIG BROTHER IS WATCHING YOU" tacked to the wall. The flat in Victory Mansions is located seven flights up, with a faulty elevator and a telescreen on the wall that cannot be completely shut off. The building is part of a setting where an economy drive is in place in preparation for Hate Week. The roof of Victory Mansions offers a view of four similar pyramidal structures, one being the Ministry of Truth, indicating it is not as grand as the ministries. The flat itself has a unique layout with the telescreen in an unusual position and a shallow alcove where Winston can sit and remain outside the screen\'s range.')